## Prepare Transformers and Emotions Data

In [ ]:
import pandas as pd

In [ ]:
# read in data
from google.colab import drive
drive.mount('/content/drive')
comments_df = pd.read_parquet("drive/MyDrive/Colab Notebooks/comments.parquet")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers
!pip install datasets
!pip install shap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import torch
import transformers
import datasets
import shap
import json

# load the emotion dataset
dataset  = datasets.load_dataset("emotion", split = "train")
data = pd.DataFrame({'text':dataset['text'],'emotion':dataset['label']})

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset emotion downloaded and prepared to /root/.cache/huggingface/datasets/emotion/split/1.0.0/cca5efe2dfeb58c1d098e0f9eeb200e9927d889b5a03c67097275dfb5fe463bd. Subsequent calls will reuse this data.


In [ ]:
# load the model and tokenizer
tokenizer = transformers.AutoTokenizer.from_pretrained("nateraw/bert-base-uncased-emotion", use_fast=True)
model = transformers.AutoModelForSequenceClassification.from_pretrained("nateraw/bert-base-uncased-emotion").cuda()

# build a pipeline object to do predictions
pred = transformers.pipeline("text-classification", model=model, tokenizer=tokenizer, device=0, return_all_scores=True)

NameError: ignored

## Configure PySpark

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
!tar xf spark-3.1.2-bin-hadoop3.2.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"
import findspark
findspark.init()

In [ ]:
!pip install sparknlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.4/489.4 kB 12.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import json

import sparknlp
import pyspark.sql.functions as F

from sparknlp.annotator import *
from sparknlp.base import *
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
from sparknlp.pretrained import PretrainedPipeline
from pyspark.sql.types import StringType, IntegerType

import pandas as pd
pd.set_option('display.max_columns',100, 'display.max_colwidth',1000, 'display.max_rows',1000, 
              'display.float_format', lambda x: '%.2f' % x)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, FloatType, StringType

# Create a SparkSession
spark = SparkSession.builder \
    .appName("BERT-type Aentiment Analysis") \
    .getOrCreate()

# Load the model and tokenizer
tokenizer = transformers.AutoTokenizer.from_pretrained("nateraw/bert-base-uncased-emotion", use_fast=True)
model = transformers.AutoModelForSequenceClassification.from_pretrained("nateraw/bert-base-uncased-emotion").cuda()

# Define a UDF (User Defined Function) to perform prediction
@udf(returnType=ArrayType(StringType()))
def predict_emotion(text):
    pred = transformers.pipeline("text-classification", model=model, tokenizer=tokenizer, device=0, return_all_scores=True)
    emo_dicts = pred(str(text))[0]
    return [i['label'] for i in emo_dicts]

# Register the UDF
spark.udf.register("predict_emotion", predict_emotion)

Py4JJavaError: ignored

In [ ]:
comments_df = comments_df.dropna()

In [ ]:
# Create a DataFrame from comments_df
comments_df = spark.createDataFrame(comments_df)

iteritems is deprecated and will be removed in a future version. Use .items instead.


In [ ]:
# Apply the UDF to each text and collect the results
comments_df = comments_df.withColumn("predicted_emotions", predict_emotion(comments_df['comment_text']))

Py4JJavaError: ignored

In [ ]:
# Define a UDF to calculate the average scores for each emotion
@udf(returnType=FloatType())
def calculate_average_scores(scores):
    if scores is not None:
        return sum(scores) / len(scores)
    else:
        return None

# Register the UDF
spark.udf.register("calculate_average_scores", calculate_average_scores)

# Explode the predicted_emotions array column and calculate average scores for each emotion
exploded_df = comments_df.selectExpr("explode(predicted_emotions) AS emotion")
comments_scores = exploded_df.groupby("emotion").agg(calculate_average_scores(exploded_df['scores']).alias("average_score"))

# Print means for each emotion
comments_scores.show()

# Stop the SparkSession
spark.stop()